In [1]:
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
import syft as sy

hook = sy.TorchHook(torch)

def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]
def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/souhail/.local/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_2.0.0.so'


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import pandas as pd
import numpy as np

def getSamples(filename):
    data = pd.read_csv(filename, sep='\t')
    return data.values[:,1:].transpose()

data1 = getSamples("GSE2034-Normal-train.txt")
data2 = getSamples("GSE2034-Tumor-train.txt")

data1Label = np.zeros(len(data1)).reshape((-1, 1))
data2Label = np.ones(len(data2)).reshape((-1, 1))
x = np.concatenate((data1, data2))
y = np.concatenate((data1Label, data2Label))

In [10]:
# We don't use the whole dataset for efficiency purpose, but feel free to increase these numbers
n_train_items = 181
n_test_items = 46

def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):#Transforms to one hot tensor
     
        onehot_tensor = torch.zeros(*index_tensor.shape, 2) # 2 Output classes
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor): #Transforms to fixed precision and secret share a tensor

        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(#Training Set)
        if i < n_train_items / args.batch_size
    ]
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(#Testing Set)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

227

In [3]:
dim = 12634

class Res1d(nn.Module):
    # the conv layers
    def __init__(self, inSize, outSize, kernel=(3,), strides=1,):
        super(Res1d, self).__init__()
        # hard-coded to do the padding correctly
        if inSize in (16,64,128,512) and strides is 2:
            pding = 0
        else:
            pding = 1
        self.l1 = nn.Conv1d(inSize, outSize, kernel, stride=strides, padding=pding, bias=False)
        self.l2 = nn.InstanceNorm1d(outSize)
        
        if strides > 1 or inSize != outSize:
            if strides > 1:
                self.r1 = nn.Identity()
                self.r2 = nn.AvgPool1d(strides)
            else:
                self.r1 = None
                self.r2 = None
            self.r3 = nn.Conv1d(inSize, outSize, 1, bias=False)
            self.r4 = nn.InstanceNorm1d(outSize)
    
        self.relu = nn.ReLU()
        
    def forward(self, x):
        l = self.l1(x)
        l = self.l2(l)
        
        if self.r1 is not None:
            r = self.r1(x)
            r = self.r2(r)
            r = self.r3(r)
            r = self.r4(r)
        else:
            r = self.r3(x)
            r = self.r4(r)
            
        x = l + r
        return self.relu(x)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(dim, 64)
        self.l2 = nn.ReLU()
        
        self.r1 = Res1d(1, 4, 3)
        
        self.r2 = Res1d(4, 8, 3)
        self.r3 = Res1d(8, 8, 3, strides=2)
        
        self.r4 = Res1d(8, 16, 3)
        self.r5 = Res1d(16, 16, 3, strides=2)
        
        self.r6 = Res1d(16, 32, 3)
        self.r7 = Res1d(32, 32, 3, strides=2)
        
        self.r8 = Res1d(32, 64, 3)
        self.r9 = Res1d(64, 64, 3, strides=2)
        
        self.r10 = Res1d(64, 128, 3)
        self.r11 = Res1d(128, 128, 3, strides=2)
        
        self.r12 = Res1d(128, 256, 3)
        self.r13 = Res1d(256, 256, 3, strides=2)
        
        self.r14 = Res1d(256, 512, 3)
        self.r15 = Res1d(512, 512, 3, strides=2)
        
        self.r16 = Res1d(512, 1024, 3)
        self.r17 = Res1d(1024, 1024, 3, strides=2)
        
        # size is by experiment and hardcode
        self.lastLinear = nn.Linear(50240,32)
        self.lastRelu = nn.ReLU()
        self.lastAgg = nn.Linear(32,1)
        self.lastSigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # shape is (batch, channel, time)
        l = x
        l = self.l2(self.l1(l))
        
        # conv layers should operate on time
        r = x
        r = self.r4(self.r3(self.r2(self.r1(r))))
        r = self.r8(self.r7(self.r6(self.r5(r))))
        r = self.r12(self.r11(self.r10(self.r9(r))))
        r = self.r16(self.r15(self.r14(self.r13(r))))
        r = self.r17(r)
        
        # flatten l
        r = r.view(x.shape[0],1, -1)
        y = torch.cat((l,r),-1)
        y = self.lastLinear(y)
        y = self.lastRelu(y)
        y = self.lastAgg(y)
        y = self.lastSigmoid(y)
        return y

In [4]:
device = torch.device("cuda")
net = Net().to(device)


In [5]:
input_test = torch.randn(80, 1, dim).float().cuda()
out = net(input_test).cuda()
out.shape

torch.Size([80, 1, 1])

# Train the model

In [6]:
# get the data into (batch, channel = 1, length=dim)
data_torch = torch.from_numpy(x).view([-1, 1, dim]).float()
label_torch = torch.from_numpy(y).view([-1,1,1]).float()

In [7]:
import torch.optim as optim
criterion = nn.BCELoss() # Binary Cross Entropy 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [8]:
for batch in range(1000):  # loop over the dataset multiple times
    # get the inputs; data is a list of [inputs, labels]
    indices = np.random.choice(len(x), size=(30))
    inputs = x[indices]
    labels = y[indices]
    
    inputs = torch.from_numpy(inputs).view([-1, 1, dim]).float().cuda()
    labels = torch.from_numpy(labels).view([-1, 1]).float().cuda()
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs).view([-1,1]).cuda()
    loss = criterion(outputs, labels).cuda()
    loss.backward()
    optimizer.step()
    
    if batch % 10 == 0:
        print(batch / 10 + 1, "% Trained")

print('Finished Training')

1.0 % Trained
2.0 % Trained
3.0 % Trained
4.0 % Trained
5.0 % Trained
6.0 % Trained
7.0 % Trained
8.0 % Trained
9.0 % Trained
10.0 % Trained
11.0 % Trained
12.0 % Trained
13.0 % Trained
14.0 % Trained
15.0 % Trained
16.0 % Trained
17.0 % Trained
18.0 % Trained
19.0 % Trained
20.0 % Trained
21.0 % Trained
22.0 % Trained
23.0 % Trained
24.0 % Trained
25.0 % Trained
26.0 % Trained
27.0 % Trained
28.0 % Trained
29.0 % Trained
30.0 % Trained
31.0 % Trained
32.0 % Trained
33.0 % Trained
34.0 % Trained
35.0 % Trained
36.0 % Trained
37.0 % Trained
38.0 % Trained
39.0 % Trained
40.0 % Trained
41.0 % Trained
42.0 % Trained
43.0 % Trained
44.0 % Trained
45.0 % Trained
46.0 % Trained
47.0 % Trained
48.0 % Trained
49.0 % Trained
50.0 % Trained
51.0 % Trained
52.0 % Trained
53.0 % Trained
54.0 % Trained
55.0 % Trained
56.0 % Trained
57.0 % Trained
58.0 % Trained
59.0 % Trained
60.0 % Trained
61.0 % Trained
62.0 % Trained
63.0 % Trained
64.0 % Trained
65.0 % Trained
66.0 % Trained
67.0 % Trained
68.0

In [9]:
# MPC learning
net

Net(
  (l1): Linear(in_features=12634, out_features=64, bias=True)
  (l2): ReLU()
  (r1): Res1d(
    (l1): Conv1d(1, 4, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (l2): InstanceNorm1d(4, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (r3): Conv1d(1, 4, kernel_size=(1,), stride=(1,), bias=False)
    (r4): InstanceNorm1d(4, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): ReLU()
  )
  (r2): Res1d(
    (l1): Conv1d(4, 8, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (l2): InstanceNorm1d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (r3): Conv1d(4, 8, kernel_size=(1,), stride=(1,), bias=False)
    (r4): InstanceNorm1d(8, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): ReLU()
  )
  (r3): Res1d(
    (l1): Conv1d(8, 8, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
    (l2): InstanceNorm1d(8, eps=1e-05, momentum=0.1, affine=False, track_runnin

In [10]:
net(data_torch.cuda())

tensor([[[4.0814e-04]],

        [[5.1038e-04]],

        [[3.8278e-04]],

        [[5.1896e-04]],

        [[2.9317e-04]],

        [[5.0704e-04]],

        [[3.5465e-04]],

        [[4.4371e-04]],

        [[2.9039e-04]],

        [[2.7970e-04]],

        [[3.3898e-04]],

        [[3.3784e-04]],

        [[4.0894e-04]],

        [[4.3664e-04]],

        [[4.4219e-04]],

        [[2.9098e-04]],

        [[3.6885e-04]],

        [[2.3485e-04]],

        [[2.8562e-04]],

        [[2.7229e-04]],

        [[3.4142e-04]],

        [[9.8182e-05]],

        [[4.3720e-04]],

        [[3.4343e-04]],

        [[3.3708e-04]],

        [[2.5448e-04]],

        [[4.7543e-04]],

        [[3.2778e-04]],

        [[4.2785e-04]],

        [[3.9049e-04]],

        [[3.3560e-04]],

        [[4.1627e-04]],

        [[1.9340e-04]],

        [[3.6553e-04]],

        [[1.3619e-04]],

        [[3.9539e-04]],

        [[1.6606e-04]],

        [[3.8498e-04]],

        [[3.8387e-04]],

        [[3.4144e-04]],

